# IMPORTS

In [2]:
from collections import *
from functools import *
from itertools import *
import sys
import re
import math
import string
import bisect
from parse import compile
import matplotlib.pyplot as plt
import numpy as np
import heapq
from matplotlib.path import Path
import ast


# SOLVING

In [74]:

def main():
    with open('big.txt', 'r') as f:
        data = f.read().splitlines()
        partitions = [-1] + [i for i, line in enumerate(data) if not line] + [len(data)]
        grids = [data[partitions[i - 1] + 1 : partitions[i]] for i in range(1, len(partitions))]
        def check_horizontal(r):
            return all(r1 == r2 for r1, r2 in zip(grid[r:], reversed(grid[:r])))
        def check_vertical(c):
            for row in grid:
                if any(c1 != c2 for c1, c2 in zip(row[c:], reversed(row[:c]))): return False
            return True
        def find_horizontal():
            for r in range(1, R):
                if check_horizontal(r): return r 
            return 0
        def find_vertical():
            for c in range(1, C):
                if check_vertical(c): return c 
            return 0
        def find_reflection(cur_r, cur_c):
            for r, c in product(range(R), range(C)):
                tmp = grid[r][c]
                grid[r][c] = "#" if tmp == "." else "."
                for i in range(1, R):
                    if check_horizontal(i) and i != cur_r: return i, 0
                for j in range(1, C):
                    if check_vertical(j) and j != cur_c: return 0, j
                grid[r][c] = tmp
            return 0, 0
        row_count = col_count = rcount = ccount = 0
        for grid in map(lambda gr: [list(line) for line in gr], grids):
            R, C = len(grid), len(grid[0])
            cur_r, cur_c = find_horizontal(), find_vertical()
            rcount += cur_r
            ccount += cur_c
            new_r, new_c = find_reflection(cur_r, cur_c)
            row_count += new_r
            col_count += new_c
        print("part 1:", ccount + 100 * rcount)
        print("part 2:", col_count + 100 * row_count)
main()

part 1: 28651
part 2: 25450


In [ ]:
25450


In [ ]:
turn_on = compile("turn on {:d},{:d} through {:d},{:d}")
turn_off = compile("turn off {:d},{:d} through {:d},{:d}")
toggle = compile("toggle {:d},{:d} through {:d},{:d}")
with open('input.txt', 'r') as f:
    data = f.read().splitlines()
    lights = [[0] * 1000 for _ in range(1000)]
    for line in data:
        to, toff, tog = turn_on.parse(line), turn_off.parse(line), toggle.parse(line)
        if to is not None:
            r1, c1, r2, c2 = to.fixed
            for r, c in product(range(r1, r2 + 1), range(c1, c2 + 1)):
                lights[r][c] += 1
        elif toff is not None:
            r1, c1, r2, c2 = toff.fixed
            for r, c in product(range(r1, r2 + 1), range(c1, c2 + 1)):
                lights[r][c] = max(0, lights[r][c] - 1)
        else:
            r1, c1, r2, c2 = tog.fixed
            for r, c in product(range(r1, r2 + 1), range(c1, c2 + 1)):
                lights[r][c] += 2
    print(sum(map(sum, lights)))